In [119]:
%load_ext dotenv
%dotenv

import os
import pandas as pd
import requests

api_key = os.getenv("SNCF_API_KEY")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# test call

In [145]:
query_date = "2022-04-30"
URL = "https://ressources.data.sncf.com//api/records/1.0/search/?dataset=tgvmax"
full_url = URL + f"&rows=-1&refine.date={query_date}"
PARAMS = {'Authentification':api_key}

r = requests.get(url = full_url, params = PARAMS)
data_json = r.json()

# test processing

In [146]:
data_json

{'nhits': 0,
 'parameters': {'dataset': ['tgvmax'],
  'rows': 10000,
  'start': 0,
  'refine': {'date': '2022-04-30'},
  'format': 'json',
  'timezone': 'UTC'},
 'records': []}

In [147]:
a  =[data_json['records'][k]['fields'] for k in range(data_json['parameters']['rows'])]

IndexError: list index out of range

In [135]:
pd.DataFrame(a)

,heure_arrivee,code_equip,date,train_no,destination,origine,entity,origine_iata,od_happy_card,axe,destination_iata,heure_depart
0,18:53,TJC,2022-04-07,4663,MONTPELLIER SAINT ROCH,MONTAUBAN VILLE BOURBON,TJOURSOSE,FRXMW,OUI,TEOZ,FRMPL,16:11
1,17:29,TJC,2022-04-07,4663,CARCASSONNE,TOULOUSE MATABIAU,TJOURSOSE,FRXYT,OUI,TEOZ,FRCCF,16:46
2,19:23,TJC,2022-04-07,4663,NIMES,TOULOUSE MATABIAU,TJOURSOSE,FRXYT,OUI,TEOZ,FRFNI,16:46
3,22:33,TJC,2022-04-07,4665,MARSEILLE ST CHARLES,AGEN,TJOURSOSE,FRAGF,OUI,TEOZ,FRMSC,17:33
4,20:12,TJC,2022-04-07,4665,BEZIERS,BORDEAUX ST JEAN,TJOURSOSE,FRBOJ,OUI,TEOZ,FRBZR,16:28
5,22:33,TJC,2022-04-07,4665,MARSEILLE ST CHARLES,BORDEAUX ST JEAN,TJOURSOSE,FRBOJ,OUI,TEOZ,FRMSC,16:28
6,20:12,TJC,2022-04-07,4665,BEZIERS,CARCASSONNE,TJOURSOSE,FRCCF,OUI,TEOZ,FRBZR,19:31
7,21:21,TJC,2022-04-07,4665,NIMES,CARCASSONNE,TJOURSOSE,FRCCF,OUI,TEOZ,FRFNI,19:31
8,22:33,TJC,2022-04-07,4665,MARSEILLE ST CHARLES,MONTPELLIER SAINT ROCH,TJOURSOSE,FRMPL,OUI,TEOZ,FRMSC,20:55
9,20:52,TJC,2022-04-07,4665,MONTPELLIER SAINT ROCH,MONTAUBAN VILLE BOURBON,TJOURSOSE,FRXMW,OUI,TEOZ,FRMPL,18:11


# day per day call

In [126]:
from datetime import datetime, timedelta

In [127]:
today = datetime.utcnow().date()

In [128]:
dfs = []
URL_BASE = "https://ressources.data.sncf.com//api/records/1.0/search/?dataset=tgvmax"
PARAMS = {'Authentification':api_key}
MAX_DAY_AHEAD = 30
q_rows = 1000
pagination = 0
data_json_empty = False
for day_ahead in range(MAX_DAY_AHEAD+1):
    query_date = today + timedelta(day_ahead)
    print(query_date)

    QUERY = f"&rows=-1&refine.date={query_date}"

    r = requests.get(url = URL_BASE + QUERY, params = PARAMS)
    data_json = r.json()
    
    try:
        if data_json['nhits'] >= 10000:
            print('more results availables')

        df_page  = pd.DataFrame([record['fields'] for record in data_json['records']])
        dfs.append(df_page)
    except Exception as exc:
        print(exc)
        break
    

2022-03-30
2022-03-31
2022-04-01
2022-04-02
2022-04-03
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-09
2022-04-10
2022-04-11
2022-04-12
2022-04-13
2022-04-14
2022-04-15
2022-04-16
2022-04-17
2022-04-18
2022-04-19
2022-04-20
2022-04-21
2022-04-22
2022-04-23
2022-04-24
2022-04-25
2022-04-26
2022-04-27
2022-04-28
2022-04-29


In [129]:
data = pd.concat(dfs)

In [130]:
data.tail()

,heure_arrivee,code_equip,date,train_no,destination,origine,entity,origine_iata,od_happy_card,axe,destination_iata,heure_depart
953,10:21,TGB,2022-04-29,9870,LORRAINE TGV,LILLE EUROPE,JCNORDEST,FRLLE,NON,EST,FREAM,8:02
954,8:53,TGB,2022-04-29,9870,AEROPORT CDG2 TGV ROISSY,LILLE EUROPE,JCNORDEST,FRLLE,NON,EST,FRMLW,8:02
955,11:01,TGB,2022-04-29,9870,STRASBOURG,AEROPORT CDG2 TGV ROISSY,JCNORDEST,FRMLW,NON,EST,FRAEG,8:59
956,9:28,TGB,2022-04-29,9870,CHAMPAGNE ARDENNE,AEROPORT CDG2 TGV ROISSY,JCNORDEST,FRMLW,NON,EST,FREAH,8:59
957,10:21,TGB,2022-04-29,9870,LORRAINE TGV,AEROPORT CDG2 TGV ROISSY,JCNORDEST,FRMLW,NON,EST,FREAM,8:59


In [148]:
data = data.sort_values('date')

In [149]:
data.to_csv('../data/exemple.csv')

In [150]:
al = data[data.origine.str.contains('PARIS')]
re = data[data.destination.str.contains('PARIS')]

In [151]:
al.to_csv('../data/aller_from_paris.csv')
re.to_csv('../data/retour_to_paris.csv')